In [2]:
import json, os, sys
%load_ext autoreload
%autoreload 2
os.environ["CUDA_VISIBLE_DEVICES"]="2"

import tensorflow as tf
from data.loader import load
from utils.session_config import setup_gpus
setup_gpus(True, 0.8)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
1 Physical GPUs, 1 Logical GPUs


In [3]:
import numpy as np
from matplotlib import pyplot as plt
from pycocotools.coco import COCO
train_ids = np.load("annotations/train_ids.npy")
test_ids = np.load("annotations/val_ids.npy")
coco = COCO("annotations/integrated_annotation.json")

loading annotations into memory...
Done (t=1.00s)
creating index...
index created!


In [8]:
def shape_process(patches, labels, masks, origins):
    patches = tf.reshape(patches, [-1, 500, 512, 3])
    patches = tf.image.resize(patches, [224,224])
    labels = tf.reshape(labels, [-1])
    masks = tf.reshape(masks, [-1, 500, 512, 3])
    origins = tf.reshape(origins, [-1, 500, 512, 3])
    masks = tf.image.resize(masks, [512,512], method='nearest')
    origins = tf.image.resize(origins, [512,512], method='nearest')
    return patches, labels, masks, origins

train_ds = load(train_ids, coco, "detection")
train_ds = train_ds.batch(8).map(shape_process).prefetch(tf.data.experimental.AUTOTUNE)
#train_ds = train_ds.cache().batch(16).map(shape_process).prefetch(tf.data.experimental.AUTOTUNE)
test_ds = load(test_ids, coco, "detection")
test_ds = test_ds.map(shape_process)

Cause: could not parse the source code of <function load.<locals>.<lambda> at 0x7f3217d5f598>: no matching AST found
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function load.<locals>.<lambda> at 0x7f3217d5f598>: no matching AST found
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function load.<locals>.<lambda> at 0x7f30e05b1488>: no matching AST found
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function load.<locals>.<lambda> at 0x7f30e05b1488>: no matching AST found
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [9]:
for patches, labels, mask, origin in train_ds.take(1):
    print(patches.shape)
    print(labels)
    print(mask.shape)
    print(origin.shape)

UnknownError: InvalidArgumentError: Input to reshape is a tensor with 3447552000 values, but the requested shape has 14592000 [Op:Reshape]
Traceback (most recent call last):

  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/script_ops.py", line 247, in __call__
    return func(device, token, args)

  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/script_ops.py", line 135, in __call__
    ret = self._func(*args)

  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/autograph/impl/api.py", line 645, in wrapper
    return func(*args, **kwargs)

  File "/home/files/cnsi-ex/data/loader.py", line 98, in data_process
    patches, labels = detection_pipeline(image_array, object_boxes, resize_H, resize_W)

  File "/home/files/cnsi-ex/data/loader.py", line 48, in detection_pipeline
    patches = tf.reshape(patches, (19, int(resize_H/10),resize_W,3))

  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/util/dispatch.py", line 206, i

# 모델 빌드 및 학습

In [ ]:
detection_model = tf.keras.models.load_model("models/detection.h5")
segmentation_model = tf.keras.models.load_model("models/segmentation.h5")

In [ ]:
CCE = tf.keras.losses.SparseCategoricalCrossentropy()

@tf.function
def detection_train_step(images, labels, model, optimizer, det_train_loss, det_train_acc):
    with tf.GradientTape() as tape:
        preds = model(images, training=True)
        loss = CCE(labels,preds)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    det_train_loss(loss)
    det_train_acc(labels, preds)
@tf.function
def detection_test_step(images, labels, model, det_test_loss, det_test_acc):
    preds = model(images, training=False)
    loss = CCE(labels,preds)
    det_test_loss(loss)
    det_test_acc(labels, preds)

In [ ]:
BCE = tf.keras.losses.BinaryCrossentropy()

@tf.function
def segmentation_train_step(images, labels, model, optimizer, seg_train_loss, seg_train_acc):
    with tf.GradientTape() as tape:
        preds = model(images, training=True)
        loss = BCE(labels,preds)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    seg_train_loss(loss)
    seg_train_acc(labels, preds)
@tf.function
def segmentation_test_step(images, labels, model, seg_test_loss, seg_test_acc):
    preds = model(images, training=False)
    loss = BCE(labels,preds)
    seg_test_loss(loss)
    seg_test_acc(labels, preds)

In [ ]:
det_train_loss = tf.keras.metrics.Mean(name='train_loss')
det_train_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
det_test_loss = tf.keras.metrics.Mean(name='test_loss')
det_test_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

seg_train_loss = tf.keras.metrics.Mean(name='train_loss')
seg_train_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
seg_test_loss = tf.keras.metrics.Mean(name='test_loss')
seg_test_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

template_det = "Epoch {} : detection train loss {:5f}, accuracy {:.3f}, val loss {:5f}, val accuracy {:.3f}"
template_seg = "Epoch {} : segmentation train loss {:5f}, accuracy {:.3f}, val loss {:5f}, val accuracy {:.3f}"

optimizer1 = tf.keras.optimizers.Adam(learning_rate=1E-3)
optimizer2 = tf.keras.optimizers.Adam(learning_rate=1E-3)
EPOCH = 20

log_dir = "logs/exp1"
summary_writer = tf.summary.create_file_writer(log_dir)

for epoch in range(1, EPOCH+1): 
    det_train_loss.reset_states()
    det_train_acc.reset_states()
    det_test_loss.reset_states()
    det_test_acc.reset_states()
    seg_train_loss.reset_states()
    seg_train_acc.reset_states()
    seg_test_loss.reset_states()
    seg_test_acc.reset_states()

    for i, (patches, labels, mask, origin) in enumerate(train_ds):
        detection_train_step(patches, labels, detection_model, optimizer1, /
                             det_train_loss, det_train_acc)
        segmentation_train_step(mask, origin, model, optimizer2,/
                                seg_train_loss, seg_train_acc)

    for i, (patches, labels, mask, origin) in enumerate(test_ds):
        detection_train_step(patches, labels, detection_model, optimizer2, /
                             det_test_loss, det_test_acc)
        segmentation_train_step(mask, origin, model, optimizer2,/
                                seg_test_loss, seg_test_acc)
        print(template.format(epoch,
                          train_loss.result().numpy(),
                          train_acc.result().numpy(),
                         test_loss.result().numpy(),
                         test_acc.result().numpy()))
        
    for imgs, labels in val_ds:
        test_step(imgs, labels, model)
    

    with summary_writer.as_default():
        tf.summary.scalar('train_loss', train_loss.result(), step=epoch)
        tf.summary.scalar('train_acc', train_acc.result().numpy(), step=epoch)
        tf.summary.scalar('test_loss', test_loss.result().numpy(), step=epoch)
        tf.summary.scalar('test_acc', test_acc.result().numpy(), step=epoch)

In [ ]:
model.save("checkpoints/detection_test_v1/model.h5")

# 테스트

In [ ]:
model = tf.keras.models.load_modelval_ds = val_ds.map(lambda x: tf.py_function(data_process, [x], [tf.float32, tf.int32])).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.map(lambda x: tf.py_function(data_process, [x], [tf.float32, tf.int32])).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.map(lambda x: tf.py_function(data_process, [x], [tf.float32, tf.int32])).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.map(lambda x: tf.py_function(data_process, [x], [tf.float32, tf.int32])).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.map(lambda x: tf.py_function(data_process, [x], [tf.float32, tf.int32])).prefetch(buffer_size=AUTOTUNE)
("checkpoints/detection_test_v1/model.h5")

In [ ]:
val_ds = tf.data.Dataset.from_tensor_slices(val_ids).cache().batch(1)
val_ds = val_ds.map(lambda x: tf.py_function(data_process, [x], [tf.float32, tf.int32])).prefetch(buffer_size=AUTOTUNE)

In [ ]:
for img, label in val_ds.take(1):
    print(img.shape)
    print(label)

In [ ]:
fig = plt.figure()

for r in range(5):
    for c in range(4):
        ax = fig.add_subplot(5,4,r*4 + c + 1)
        if r*4+c+1>19:
            break
        ax.imshow((img[r*4+c]+1)/2)

In [ ]:
model(img)